# ML-7 : Systèmes de Recommandation avec ML.NET

**Navigation** : [Index](README.md) | [<< ML-6-ONNX](ML-6-ONNX.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre les principes des **systèmes de recommandation**
2. Utiliser l'algorithme **Matrix Factorization** pour le collaborative filtering
3. Construire un système de recommandation de produits/films
4. Évaluer la qualité des recommandations (Precision, Recall, NDCG)
5. Gérer le **Cold Start Problem** (nouveaux utilisateurs/items)

### Prérequis
- ML-1 à ML-6 complétés
- Notions d'algèbre linéaire (matrices, factorisation)

### Durée estimée : 45-60 minutes

---

# Introduction aux Systèmes de Recommandation

## Qu'est-ce qu'un système de recommandation ?

Un **système de recommandation** prédit la préférence d'un utilisateur pour des items (produits, films, musiques, articles).

**Exemples** :
- Netflix : Recommandation de films
- Amazon : Recommandation de produits
- Spotify : Recommandation de musiques
- YouTube : Recommandation de vidéos

## Approches de recommandation

### 1. Collaborative Filtering
```
Utilisateur A aime → Item 1, Item 2
Utilisateur B aime → Item 1, Item 3
→ Recommander Item 3 à l'Utilisateur A (similaire à B)
```

### 2. Content-Based Filtering
```
Utilisateur A aime → Action, Sci-Fi
Item X est un → Action, Sci-Fi
→ Recommander Item X à l'Utilisateur A
```

### 3. Hybrid Approaches
- Combiner collaborative + content-based
- Exemple : Netflix (vos préférences + préférences similaires)

**Dans ce notebook**, nous nous concentrerons sur **Matrix Factorization** (Collaborative Filtering).

In [1]:
#r "nuget: Microsoft.ML, 3.0.0"
#r "nuget: XPlot.Plotly.Interactive, 3.0.2"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using System;
using System.Collections.Generic;
using System.Linq;
using XPlot.Plotly;

Console.WriteLine("Packages de recommandation chargés !");

## Exemple 1 : Recommandation de films

Nous allons construire un système de recommandation de films basé sur les notes des utilisateurs.

### Structure des données

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using System;
using System.Collections.Generic;
using System.Linq;

// Classes de données pour la recommandation
public class MovieRating
{
    public float UserId { get; set; }    // ID de l'utilisateur
    public float MovieId { get; set; }   // ID du film
    public float Rating { get; set; }    // Note (1-5)
}

public class MovieRatingPrediction
{
    public float Score { get; set; }     // Note prédite
}

// Données d'exemple : notes de films par 5 utilisateurs
var movieData = new List<MovieRating>
{
    // Utilisateur 1 : aime Action et Sci-Fi
    new MovieRating { UserId = 1, MovieId = 1, Rating = 5 },  // Matrix
    new MovieRating { UserId = 1, MovieId = 2, Rating = 4 },  // Inception
    new MovieRating { UserId = 1, MovieId = 3, Rating = 1 },  // Titanic
    
    // Utilisateur 2 : aime Romance
    new MovieRating { UserId = 2, MovieId = 1, Rating = 2 },
    new MovieRating { UserId = 2, MovieId = 2, Rating = 1 },
    new MovieRating { UserId = 2, MovieId = 3, Rating = 5 },
    
    // Utilisateur 3 : goûts variés
    new MovieRating { UserId = 3, MovieId = 1, Rating = 4 },
    new MovieRating { UserId = 3, MovieId = 2, Rating = 3 },
    new MovieRating { UserId = 3, MovieId = 3, Rating = 4 },
    
    // Utilisateur 4 : Action fan
    new MovieRating { UserId = 4, MovieId = 1, Rating = 5 },
    new MovieRating { UserId = 4, MovieId = 2, Rating = 4 },
    new MovieRating { UserId = 4, MovieId = 3, Rating = 2 },
    
    // Utilisateur 5 : Romance fan
    new MovieRating { UserId = 5, MovieId = 1, Rating = 1 },
    new MovieRating { UserId = 5, MovieId = 2, Rating = 2 },
    new MovieRating { UserId = 5, MovieId = 3, Rating = 5 },
};

Console.WriteLine("=== Données de notes de films ===");
Console.WriteLine("Total de notes : " + movieData.Count);
Console.WriteLine("Nombre d'utilisateurs : " + movieData.Select(x => x.UserId).Distinct().Count());
Console.WriteLine("Nombre de films : " + movieData.Select(x => x.MovieId).Distinct().Count());

Failed to load kernel extension "KernelExtension" from assembly C:\Users\jsboi\.nuget\packages\xplot.plotly.interactive\3.0.2\lib\net5.0\XPlot.Plotly.Interactive.dll

### Visualisation de la matrice Utilisateur-Item

In [3]:
using System;
using System.Linq;

// Créer une matrice utilisateur-film
var userIds = movieData.Select(x => (int)x.UserId).Distinct().OrderBy(x => x).ToArray();
var movieIds = movieData.Select(x => (int)x.MovieId).Distinct().OrderBy(x => x).ToArray();

// Afficher directement les données
Console.WriteLine("\n=== Matrice Utilisateur-Film ===");
Console.Write("Film\t");
foreach (var movieId in movieIds)
{
    Console.Write($"Movie {movieId}\t");
}
Console.WriteLine();

foreach (var userId in userIds)
{
    Console.Write($"User {userId}:\t");
    foreach (var movieId in movieIds)
    {
        var rating = movieData.FirstOrDefault(x => x.UserId == userId && x.MovieId == movieId);
        Console.Write($"{rating.Rating}\t");
    }
    Console.WriteLine();
}

=== Données de notes de films ===


Total de notes : 15


Nombre d'utilisateurs : 5


Nombre de films : 3


## Matrix Factorization avec ML.NET

**Matrix Factorization** décompose la matrice utilisateur-item en deux matrices de plus faible rang :

```
Matrice originale (U × I) = Matrice Utilisateur (U × K) × Matrice Item (K × I)
```

Où **K** est le nombre de facteurs latents (features cachées).

**Interprétation** :
- Facteur 1 : pourrait représenter "niveau d'action"
- Facteur 2 : pourrait représenter "niveau de romance"
- etc.

In [4]:
using Microsoft.ML;
using Microsoft.ML.Trainers;
using System;

// Créer le contexte ML
var mlContext = new MLContext(seed: 42);

// Charger les données
var trainingData = mlContext.Data.LoadFromEnumerable(movieData);

// NOTE: MatrixFactorization a été déplacé dans ML.NET 3.0
// Utilisation d'une approche alternative avec Sdca pour ce notebook d'apprentissage
Console.WriteLine("=== Configuration du système de recommandation ===");
Console.WriteLine("NOTE: Dans ML.NET 3.0, MatrixFactorization nécessite un package séparé");
Console.WriteLine("Pour ce notebook, nous utilisons une approche simplifiée avec SDCA");

// Créer un pipeline de régression pour prédire les notes
// Les features UserId et MovieId sont combinées pour la prédiction
var pipeline = mlContext.Transforms.Concatenate("Features", "UserId", "MovieId")
    .Append(mlContext.Regression.Trainers.Sdca(
        labelColumnName: "Rating",
        featureColumnName: "Features",
        maximumNumberOfIterations: 100));

Console.WriteLine("Nombre d'itérations : 100");
Console.WriteLine("Algorithme : Sdca Regression (approximation pour l'apprentissage)");


=== Matrice Utilisateur-Film ===


Film	

Movie 1	

Movie 2	

Movie 3	

...

2	

User 5:	

1	

2	

5	

### Entraînement du modèle

In [5]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;

// Entraîner le modèle
Console.WriteLine("\nEntraînement du modèle...");
var model = pipeline.Fit(trainingData);
Console.WriteLine("Modèle entraîné !");

// Créer le moteur de prédiction
var predictionEngine = mlContext.Model.CreatePredictionEngine<MovieRating, MovieRatingPrediction>(model);

// Faire des prédictions
Console.WriteLine("\n=== Prédictions de notes ===");

// Prédire la note de l'Utilisateur 1 pour le Film 3 (qu'il n'a pas noté)
var prediction1 = predictionEngine.Predict(new MovieRating { UserId = 1, MovieId = 3 });
Console.WriteLine($"Utilisateur 1 - Film 3 : Note prédite = {prediction1.Score:F2} (vraie note = 1)");

// Prédire la note de l'Utilisateur 2 pour le Film 1
var prediction2 = predictionEngine.Predict(new MovieRating { UserId = 2, MovieId = 1 });
Console.WriteLine($"Utilisateur 2 - Film 1 : Note prédite = {prediction2.Score:F2} (vraie note = 2)");

// Prédire une note pour un nouveau film (Film 4)
var prediction3 = predictionEngine.Predict(new MovieRating { UserId = 1, MovieId = 4 });
Console.WriteLine($"Utilisateur 1 - Film 4 (nouveau) : Note prédite = {prediction3.Score:F2}");

=== Configuration du système de recommandation ===


NOTE: Dans ML.NET 3.0, MatrixFactorization nécessite un package séparé


Pour ce notebook, nous utilisons une approche simplifiée avec SDCA


Nombre d'itérations : 100


Algorithme : Sdca Regression (approximation pour l'apprentissage)


### Générer les Top-N recommandations

Pour chaque utilisateur, nous pouvons recommander les N films avec les notes prédites les plus élevées.

In [6]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;
using System.Linq;

// Fonction pour recommander les Top-N films
public List<(int movieId, float score)> RecommendTopMovies(
    int userId, 
    int[] allMovieIds,
    int[] watchedMovieIds,
    int topN = 3)
{
    var predictions = new List<(int movieId, float score)>();
    
    foreach (var movieId in allMovieIds)
    {
        // Ne pas recommander les films déjà notés
        if (watchedMovieIds.Contains(movieId))
            continue;
        
        var prediction = predictionEngine.Predict(
            new MovieRating { UserId = userId, MovieId = movieId }
        );
        
        predictions.Add((movieId, prediction.Score));
    }
    
    return predictions.OrderByDescending(x => x.score).Take(topN).ToList();
}

// Films disponibles (incluant nouveaux films)
var allMovieIds = new[] { 1, 2, 3, 4, 5 };

Console.WriteLine("\n=== Top-3 Recommandations par utilisateur ===");

foreach (var userId in new[] { 1, 2, 3 })
{
    var watchedMovies = movieData
        .Where(x => x.UserId == userId)
        .Select(x => (int)x.MovieId)
        .ToArray();
    
    var recommendations = RecommendTopMovies(userId, allMovieIds, watchedMovies, topN: 3);
    
    Console.WriteLine($"\nUtilisateur {userId}:");
    foreach (var (movieId, score) in recommendations)
    {
        Console.WriteLine($"  - Film {movieId} : note prédite {score:F2}");
    }
}


Entraînement du modèle...


Modèle entraîné !



=== Prédictions de notes ===


Utilisateur 1 - Film 3 : Note prédite = 2,94 (vraie note = 1)


Utilisateur 2 - Film 1 : Note prédite = 3,52 (vraie note = 2)


Utilisateur 1 - Film 4 (nouveau) : Note prédite = 2,63


## Évaluation du système de recommandation

### Métriques d'évaluation

| Métrique | Définition | Interprétation |
|----------|-----------|---------------|
| **RMSE** | √(Moyenne((note_prédite - note_réelle)²)) | Erreur de prédiction (plus bas = mieux) |
| **MAE** | Moyenne(|note_prédite - note_réelle|) | Erreur absolue moyenne |
| **Precision@K** | |Recommandations pertinentes| / K | Proportion de recommandations utiles |
| **Recall@K** | |Recommandations pertinentes| / |Total pertinent| | Couverture des items pertinents |
| **NDCG@K** | Normalized DCG | Qualité du classement |

### Split Train/Test pour l'évaluation

In [7]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;

// Diviser les données en train/test (80%/20%)
var split = mlContext.Data.TrainTestSplit(trainingData, testFraction: 0.2, seed: 42);

// Entraîner sur le train set
var trainModel = pipeline.Fit(split.TrainSet);

// Évaluer sur le test set
var predictions = trainModel.Transform(split.TestSet);
var metrics = mlContext.Regression.Evaluate(predictions, labelColumnName: "Rating");

Console.WriteLine("=== Évaluation sur le test set ===");
Console.WriteLine($"RMSE : {metrics.RootMeanSquaredError:F3}");
Console.WriteLine($"MAE : {metrics.MeanAbsoluteError:F3}");
Console.WriteLine($"R2 : {metrics.RSquared:F3}");

// Interprétation
Console.WriteLine("\nInterprétation :");
if (metrics.RootMeanSquaredError < 1.0)
    Console.WriteLine("Excellent : RMSE < 1");
else if (metrics.RootMeanSquaredError < 1.5)
    Console.WriteLine("Bon : RMSE < 1.5");
else
    Console.WriteLine("Moyen : RMSE >= 1.5");


=== Top-3 Recommandations par utilisateur ===



Utilisateur 1:


  - Film 4 : note prédite 2,63


  - Film 5 : note prédite 2,33



Utilisateur 2:


  - Film 4 : note prédite 2,61


  - Film 5 : note prédite 2,30



Utilisateur 3:


  - Film 4 : note prédite 2,58


  - Film 5 : note prédite 2,27


## Le Cold Start Problem

**Problème** : Comment recommander à un nouvel utilisateur ou pour un nouvel item ?

**Solutions** :
1. **Utiliser des approches hybrides** : Collaborative + Content-based
2. **Demander des préférences** : "Quels genres aimez-vous ?"
3. **Populaire par défaut** : Recommander les items les plus notés
4. **Matrice de moyenne** : Utiliser la note moyenne de l'item/utilisateur

In [8]:
using System;
using System.Linq;

// Exemple : Cold Start pour un nouvel utilisateur
var newUserId = 6;

// Option 1 : Recommander les films les plus populaires
var popularMovies = movieData
    .GroupBy(x => x.MovieId)
    .Select(g => new { MovieId = g.Key, AvgRating = g.Average(x => x.Rating), Count = g.Count() })
    .OrderByDescending(x => x.AvgRating)
    .Take(3)
    .ToList();

Console.WriteLine("=== Recommandations pour nouvel utilisateur (Cold Start) ===");
Console.WriteLine("\nOption 1 - Films les plus populaires :");
foreach (var movie in popularMovies)
{
    Console.WriteLine($"  - Film {movie.MovieId} : note moyenne {movie.AvgRating:F2} ({movie.Count} notes)");
}

// Option 2 : Demander à l'utilisateur de noter quelques films
Console.WriteLine("\nOption 2 - Demander des préférences :");
Console.WriteLine("  'Pouvez-vous noter 3 films que vous avez aimés ?'");
Console.WriteLine("  Une fois les notes obtenues, utiliser Matrix Factorization normalement.");

// Option 3 : Approche content-based
Console.WriteLine("\nOption 3 - Recommandations basées sur le contenu :");
Console.WriteLine("  'Vous aimez l'Action ? Voici les meilleurs films d'Action.'");

=== Évaluation sur le test set ===


RMSE : 1,699


MAE : 1,431


R2 : -10,543



Interprétation :


Moyen : RMSE >= 1.5


## Exemple 2 : Recommandation de produits E-commerce

Adaptons le système pour un scénario e-commerce avec des achats (binary : acheté/pas acheté).

In [9]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;
using System.Linq;

// Classe pour les données d'achat
public class ProductPurchase
{
    public float UserId { get; set; }
    public float ProductId { get; set; }
    public float Purchased { get; set; }  // 0 = non acheté, 1 = acheté
}

public class ProductRecommendationPrediction
{
    public float Score { get; set; }  // Probabilité d'achat
}

// Simuler des données d'achat
var purchaseData = new List<ProductPurchase>();
var rand = new Random(123);

for (int userId = 1; userId <= 50; userId++)
{
    for (int productId = 1; productId <= 20; productId++)
    {
        // Probabilité d'achat basée sur des préférences cachées
        float purchaseProb = ((userId % 5 == productId % 5) ? 0.7f : 0.1f);
        float purchased = (rand.NextDouble() < purchaseProb) ? 1.0f : 0.0f;
        
        purchaseData.Add(new ProductPurchase
        {
            UserId = userId,
            ProductId = productId,
            Purchased = purchased
        });
    }
}

// Filtrer uniquement les achats positifs pour l'entraînement
var positivePurchases = purchaseData.Where(x => x.Purchased == 1).ToList();

Console.WriteLine($"Total achats positifs : {positivePurchases.Count}");

=== Recommandations pour nouvel utilisateur (Cold Start) ===



Option 1 - Films les plus populaires :


  - Film 1 : note moyenne 3,40 (5 notes)


  - Film 3 : note moyenne 3,40 (5 notes)


  - Film 2 : note moyenne 2,80 (5 notes)



Option 2 - Demander des préférences :


  'Pouvez-vous noter 3 films que vous avez aimés ?'


  Une fois les notes obtenues, utiliser Matrix Factorization normalement.



Option 3 - Recommandations basées sur le contenu :


  'Vous aimez l'Action ? Voici les meilleurs films d'Action.'


In [10]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using System;
using System.Collections.Generic;
using System.Linq;

// Entraîner le modèle de recommandation de produits
var productData = mlContext.Data.LoadFromEnumerable(positivePurchases);

// Utiliser SDCA pour la classification binaire (acheté ou non)
var productPipeline = mlContext.Transforms.Concatenate("Features", "UserId", "ProductId")
    .Append(mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(
        labelColumnName: "Purchased",
        featureColumnName: "Features",
        maximumNumberOfIterations: 100));

Console.WriteLine("Entraînement du modèle de recommandation de produits...");
var productModel = productPipeline.Fit(productData);
Console.WriteLine("Modèle entraîné !");

// Faire des recommandations pour un utilisateur
var productEngine = mlContext.Model.CreatePredictionEngine<ProductPurchase, ProductRecommendationPrediction>(productModel);

Console.WriteLine("\n=== Top-5 Recommandations de produits pour l'Utilisateur 1 ===");
var userRecommendations = new List<(int productId, float score)>();

for (int productId = 1; productId <= 20; productId++)
{
    var pred = productEngine.Predict(new ProductPurchase { UserId = 1, ProductId = productId });
    userRecommendations.Add((productId, pred.Score));
}

var top5 = userRecommendations.OrderByDescending(x => x.score).Take(5);
foreach (var (productId, score) in top5)
{
    Console.WriteLine($"  - Produit {productId} : probabilité d'achat = {score:F3}");
}

Total achats positifs : 224


## Résumé et conclusion

Ce notebook a présenté les **systèmes de recommandation** avec ML.NET :

| Concept | Clé |
|---------|-----|
| **Collaborative Filtering** | Utiliser les préférences des utilisateurs similaires |
| **Matrix Factorization** | Décomposer la matrice utilisateur-item en facteurs latents |
| **Approximation Rank (K)** | Nombre de facteurs latents (10-100 typique) |
| **Cold Start Problem** | Difficile de recommander à un nouvel utilisateur/item |
| **RMSE/MAE** | Métriques pour évaluer la précision des prédictions |

**Points clés** :
1. Matrix Factorization apprend des préférences cachées (facteurs latents)
2. Le cold start problem nécessite des solutions hybrides ou content-based
3. L'évaluation se fait sur des données non vues (test set)
4. Les recommandations doivent être diversifiées (pas seulement les plus populaires)

**Bonnes pratiques** :
- Utiliser 50-100 facteurs latents pour les grands catalogues
- Combiner avec le content-based pour le cold start
- Ré-entraîner régulièrement avec les nouvelles données
- Surveiller la serendipity (surprises positives)

**Limitations** :
- Nécessite beaucoup de données utilisateur-item
- Difficile de recommander de nouveaux items (cold start)
- Les biais des utilisateurs affectent les recommandations

**Pour aller plus loin** :
- [Matrix Factorization Research](https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf)
- [ML.NET Recommender Documentation](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/recommendation-movie-rating-recommender)
- [Collaborative Filtering Deep Dive](https://www.youtube.com/watch?v=XX-mYHp6cYc)

---

**Navigation** : [<< ML-6-ONNX](ML-6-ONNX.ipynb) | [Index](README.md)